In [1]:
import pandas as pd
pd.options.display.max_columns = None
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.impute import KNNImputer
from scipy.cluster.hierarchy import dendrogram
import numpy as np
from io import StringIO
import warnings
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

In [3]:
datistrutturali=pd.read_excel('../data/1_input/1_structural/DatiStrutturali_1822_float.xlsx')
datistrutturali.dtypes # tipi di dati per colonna
print(datistrutturali.describe()) # descrizione statistica dei dati
print(datistrutturali.isnull().sum(axis=0)) # numero di NaN per dato
print(len(datistrutturali.columns))

               HOUR           DAY          WEEK         MONTH          YEAR  \
count  43824.000000  43824.000000  43824.000000  43824.000000  43824.000000   
mean      12.500000    183.100219     26.593556      6.523549   2020.000000   
std        6.922266    105.425347     15.061517      3.448572      1.413842   
min        1.000000      1.000000      1.000000      1.000000   2018.000000   
25%        6.750000     92.000000     14.000000      4.000000   2019.000000   
50%       12.500000    183.000000     27.000000      7.000000   2020.000000   
75%       18.250000    274.000000     40.000000     10.000000   2021.000000   
max       24.000000    366.000000     54.000000     12.000000   2022.000000   

            Biomass    FossilCoal     FossilGas     FossilOil         Hydro  \
count  43824.000000  43824.000000  43824.000000  43824.000000  43824.000000   
mean     195.369980    403.297782   6722.441881      7.695555   3839.969081   
std       41.471803    414.994423   3200.077678    

Some values are NA, for North-CNorth capacity and exchanges

In [4]:
datistrutturali[datistrutturali.isna().any(axis=1)]

,HOUR,DAY,WEEK,MONTH,YEAR,Biomass,FossilCoal,FossilGas,FossilOil,Hydro,LoadForecast_nord,SECONDARIA,TERZIARIA_TOTALEup,TERZIARIAdw,CommNord_Cnord,CommCnord_Nord,CommAT_Nord,CommNord_AT,CommCH_Nord,CommNord_CH,CommFR_Nord,CommNord_FR,CommSl_Nord,CommNord_Sl,Pvforecast,CapNord_Cnord,CapCnord_Nord,CapAT_Nord,CapNord_AT,CapCH_Nord,CapNord_CH,CapFR_Nord,CapNord_FR,CapSl_Nord,CapNord_Sl,WaterReservoirs,WINDforecast
27816,1,64,10,3,2021,240,29,6660,2,1853,15516,271.497331,846.769070,846.769070,1925.0,0.0,150,0,2782.0,0.0,1624,0,631,0,0,NaN,NaN,150,145,3696,1910,1624,1160,631,680,1068286,20
27817,2,64,10,3,2021,241,28,6479,2,1644,14692,261.606608,781.872303,781.872303,1681.0,0.0,150,0,2998.0,36.0,1624,0,580,0,0,NaN,NaN,150,145,3696,1910,1624,1160,631,680,1068286,20
27818,3,64,10,3,2021,237,29,6277,2,1507,14401,258.056369,743.411224,743.411224,1201.0,0.0,150,0,2603.0,5.0,1624,0,610,0,0,NaN,NaN,150,145,3696,1910,1624,1160,631,680,1068286,20
27819,4,64,10,3,2021,236,29,5979,2,1483,14373,257.713134,720.247260,720.247260,886.0,0.0,150,0,2351.0,1.0,1624,0,631,0,0,NaN,NaN,150,145,3696,1910,1624,1160,631,680,1068286,20
27820,5,64,10,3,2021,241,29,6517,2,1534,14545,259.817032,712.814956,712.814956,465.0,0.0,150,0,2379.0,1.0,1624,0,534,0,0,NaN,NaN,150,145,3696,1910,1624,1160,631,680,1068286,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40938,19,245,35,9,2022,175,366,10999,5,5806,21548,337.831938,1670.837707,1701.046674,NaN,NaN,264,0,2181.0,0.0,1629,0,494,0,83,3800.0,2756.0,264,80,2181,1440,2629,870,494,620,1303079,0
40939,20,245,35,9,2022,176,366,11184,5,5191,21347,335.767434,1591.824823,1591.843689,NaN,NaN,264,0,2181.0,0.0,2043,0,494,0,0,3800.0,2756.0,264,80,2181,1440,2629,870,494,620,1303079,0
40940,21,245,35,9,2022,178,365,11373,15,3918,19882,320.446596,1456.166015,1456.186639,NaN,NaN,255,0,1768.0,0.0,981,0,473,0,0,3846.0,2859.0,255,80,2095,1440,2518,870,473,620,1303079,0
40941,22,245,35,9,2022,177,364,11435,15,2627,18315,303.486494,1356.096406,1356.118552,NaN,NaN,255,0,1381.0,75.0,0,251,473,0,0,3746.0,3059.0,255,80,2095,1440,2518,870,473,620,1303079,0


Let's impute the value corresponding to the k-nearest neighbor

In [5]:
imputer = KNNImputer()
res = imputer.fit_transform(datistrutturali)
datistrutturali = pd.DataFrame(res, columns=imputer.feature_names_in_, index=datistrutturali.index)
datistrutturali

,HOUR,DAY,WEEK,MONTH,YEAR,Biomass,FossilCoal,FossilGas,FossilOil,Hydro,LoadForecast_nord,SECONDARIA,TERZIARIA_TOTALEup,TERZIARIAdw,CommNord_Cnord,CommCnord_Nord,CommAT_Nord,CommNord_AT,CommCH_Nord,CommNord_CH,CommFR_Nord,CommNord_FR,CommSl_Nord,CommNord_Sl,Pvforecast,CapNord_Cnord,CapCnord_Nord,CapAT_Nord,CapNord_AT,CapCH_Nord,CapNord_CH,CapFR_Nord,CapNord_FR,CapSl_Nord,CapNord_Sl,WaterReservoirs,WINDforecast
0,1.0,1.0,1.0,1.0,2018.0,113.0,344.0,1543.0,0.0,592.0,11298.0,218.076079,872.236657,872.236657,0.0,464.0,139.0,0.0,1356.0,50.0,1285.0,0.0,298.0,0.0,0.0,3700.0,2500.0,139.0,145.0,1778.0,1910.0,1285.0,1160.0,298.0,680.0,2114229.0,3.0
1,2.0,1.0,1.0,1.0,2018.0,151.0,465.0,2071.0,0.0,681.0,10502.0,207.099426,792.443511,792.443511,0.0,168.0,139.0,0.0,1371.0,0.0,1285.0,0.0,298.0,0.0,0.0,3700.0,2500.0,139.0,145.0,1778.0,1910.0,1285.0,1160.0,298.0,680.0,2114229.0,4.0
2,3.0,1.0,1.0,1.0,2018.0,112.0,342.0,1349.0,0.0,566.0,9983.0,199.757059,732.900085,732.900085,0.0,614.0,139.0,0.0,1221.0,36.0,1285.0,0.0,298.0,0.0,0.0,3700.0,2500.0,139.0,145.0,1778.0,1910.0,1285.0,1160.0,298.0,680.0,2114229.0,4.0
3,4.0,1.0,1.0,1.0,2018.0,152.0,438.0,1566.0,0.0,374.0,9664.0,195.166626,689.309898,689.309898,0.0,961.0,139.0,0.0,1044.0,0.0,1285.0,0.0,298.0,0.0,0.0,3700.0,2500.0,139.0,145.0,1778.0,1910.0,1285.0,1160.0,298.0,680.0,2114229.0,5.0
4,5.0,1.0,1.0,1.0,2018.0,153.0,419.0,1633.0,0.0,-61.0,9648.0,194.934776,669.995099,669.995099,0.0,1300.0,125.0,0.0,1243.0,0.0,1076.0,0.0,249.0,0.0,0.0,3700.0,2500.0,125.0,145.0,1548.0,1910.0,1076.0,1160.0,249.0,680.0,2114229.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,20.0,365.0,53.0,12.0,2022.0,243.0,534.0,8390.0,2.0,3142.0,14533.0,259.670599,1184.839134,1184.865941,2796.0,0.0,325.0,0.0,1507.0,0.0,2028.0,0.0,0.0,660.0,0.0,3700.0,3002.0,325.0,100.0,1507.0,1810.0,3885.0,1495.0,753.0,660.0,1284533.0,2.0
43820,21.0,365.0,53.0,12.0,2022.0,244.0,534.0,7678.0,2.0,2402.0,13343.0,244.879729,1065.506754,1065.536564,1921.0,0.0,325.0,0.0,1507.0,0.0,1577.0,0.0,0.0,209.0,0.0,3600.0,3102.0,325.0,100.0,1507.0,1810.0,3885.0,1495.0,753.0,660.0,1284533.0,2.0
43821,22.0,365.0,53.0,12.0,2022.0,244.0,535.0,6733.0,2.0,1810.0,12190.0,230.000000,961.114553,961.147600,1314.0,0.0,0.0,100.0,1507.0,0.0,2453.0,0.0,0.0,660.0,0.0,3749.0,3220.0,325.0,100.0,1507.0,1810.0,3885.0,1495.0,753.0,660.0,1284533.0,2.0
43822,23.0,365.0,53.0,12.0,2022.0,237.0,533.0,6313.0,2.0,1437.0,11187.0,216.565137,880.729844,880.765908,1349.0,0.0,0.0,100.0,1459.0,0.0,2401.0,0.0,0.0,660.0,0.0,3749.0,3220.0,325.0,100.0,1459.0,1810.0,3885.0,1495.0,753.0,660.0,1284533.0,2.0


In [6]:
## Correlazione tra le variabili
corr=datistrutturali.corr(method='pearson')
fig = go.Figure(data=go.Heatmap(z=corr.values,x=corr.index,y=corr.columns,hoverongaps = False))
#fig.show()
## Distribuzione dei dati
#col='Biomass'
#dati=datistrutturali[col].fillna(datistrutturali[col].mean())
#ff.create_distplot([dati],[col])
#fig.show()

In [7]:
fig.show()

In [8]:
dates = pd.date_range('01/01/2018', periods=43824, freq='H')

pro_cons_col=['Biomass','FossilCoal','FossilGas','FossilOil','Hydro','LoadForecast_nord','WaterReservoirs','WINDforecast', 'Pvforecast']
pro_cons=pd.DataFrame(index=dates,columns=[i for i in pro_cons_col])
for i in pro_cons_col:
    pro_cons[i]=datistrutturali[i].values

res_col=['SECONDARIA','TERZIARIA_TOTALEup','TERZIARIAdw']
res=pd.DataFrame(index=dates,columns=[i for i in res_col])
for i in res_col:
    res[i]=datistrutturali[i].values

scamb_comm_col=['CommNord_Cnord','CommCnord_Nord', 'CommAT_Nord', 'CommNord_AT', 'CommCH_Nord','CommNord_CH', 'CommFR_Nord','CommNord_FR', 'CommSl_Nord', 'CommNord_Sl']
scamb_comm=pd.DataFrame(index=dates,columns=[i for i in scamb_comm_col])
for i in scamb_comm_col:
    scamb_comm[i]=datistrutturali[i].values

cap_col=['CapNord_Cnord','CapCnord_Nord', 'CapAT_Nord', 'CapNord_AT', 'CapCH_Nord', 'CapNord_CH', 'CapFR_Nord', 'CapNord_FR','CapSl_Nord', 'CapNord_Sl']
cap=pd.DataFrame(index=dates,columns=[i for i in cap_col])
for i in cap_col:
    cap[i]=datistrutturali[i].values
    
tutti=pd.DataFrame(index=dates,columns=(scamb_comm_col+cap_col+pro_cons_col+res_col))
for i in scamb_comm_col+cap_col+pro_cons_col+res_col:
    tutti[i]=datistrutturali[i].values
    
perimetro=pd.DataFrame(index=dates,columns=(scamb_comm_col+cap_col))
for i in scamb_comm_col+cap_col:
    perimetro[i]=datistrutturali[i].values

interno=pd.DataFrame(index=dates,columns=(pro_cons_col+res_col))
for i in pro_cons_col+res_col:
    interno[i]=datistrutturali[i].values

tre_a=pd.DataFrame(index=dates,columns=(scamb_comm_col+pro_cons_col))
for i in scamb_comm_col+pro_cons_col:
    tre_a[i]=datistrutturali[i].values

tre_b=pd.DataFrame(index=dates,columns=(res_col+cap_col))
for i in res_col+cap_col:
    tre_b[i]=datistrutturali[i].values

quat_a=pd.DataFrame(index=dates,columns=scamb_comm_col)
for i in scamb_comm_col:
    quat_a[i]=datistrutturali[i].values

quat_b=pd.DataFrame(index=dates,columns=(pro_cons_col+res_col))
for i in pro_cons_col+res_col:
    quat_b[i]=datistrutturali[i].values

quat_c=pd.DataFrame(index=dates,columns=cap_col)
for i in cap_col:
    quat_c[i]=datistrutturali[i].values
    
cin_c=pd.DataFrame(index=dates,columns=pro_cons_col+res_col+scamb_comm_col)
for i in pro_cons_col+res_col+scamb_comm_col:
    cin_c[i]=datistrutturali[i].values
    
sei_c=pd.DataFrame(index=dates,columns=pro_cons_col+res_col+cap_col)
for i in pro_cons_col+res_col+cap_col:
    sei_c[i]=datistrutturali[i].values
    
## Scaling the dataset (mean zero, std dev one)
pro_cons_std=pd.DataFrame(StandardScaler().fit_transform(pro_cons),index=dates,columns=pro_cons_col)
res_std=pd.DataFrame(StandardScaler().fit_transform(res),index=dates,columns=res_col)
sccomm_std=pd.DataFrame(StandardScaler().fit_transform(scamb_comm),index=dates,columns=scamb_comm_col)
cap_std=pd.DataFrame(StandardScaler().fit_transform(cap),index=dates,columns=cap_col)
tutti_std=pd.DataFrame(StandardScaler().fit_transform(tutti),index=dates,columns=scamb_comm_col+cap_col+pro_cons_col+res_col)
perimetro_std=pd.DataFrame(StandardScaler().fit_transform(perimetro),index=dates,columns=scamb_comm_col+cap_col)
interno_std=pd.DataFrame(StandardScaler().fit_transform(interno),index=dates,columns=pro_cons_col+res_col)
tre_a_std=pd.DataFrame(StandardScaler().fit_transform(tre_a),index=dates,columns=scamb_comm_col+pro_cons_col)
tre_b_std=pd.DataFrame(StandardScaler().fit_transform(tre_b),index=dates,columns=res_col+cap_col)
quat_a_std=pd.DataFrame(StandardScaler().fit_transform(quat_a),index=dates,columns=scamb_comm_col)
quat_b_std=pd.DataFrame(StandardScaler().fit_transform(quat_b),index=dates,columns=pro_cons_col+res_col)
quat_c_std=pd.DataFrame(StandardScaler().fit_transform(quat_c),index=dates,columns=cap_col)
cin_c_std=pd.DataFrame(StandardScaler().fit_transform(cin_c),index=dates,columns=pro_cons_col+res_col+scamb_comm_col)
sei_c_std=pd.DataFrame(StandardScaler().fit_transform(sei_c),index=dates,columns=pro_cons_col+res_col+cap_col)

#scaler=StandardScaler().fit(dati) #compute the mean and the standard deviation to be used for scaling
#datistd=scaler.transform(dati) # perform standardization by centering and scaling 
#print(scaler.mean_)
#print(dati) # ogni elemento [] dell'array è una riga, non una colonna


In [9]:
# (1) 
# Tutti i dati
pca_all = PCA(n_components=5)
prcomp_tutti=pca_all.fit(tutti_std.values) # componenti principali
scores_tutti = pca_all.fit_transform(tutti_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVall=prcomp_tutti.explained_variance_ratio_
EVall_cum=np.cumsum(EVall)

print("EVall_cum: ")
print(EVall_cum)


EVall_cum: 
[0.17357458 0.3345615  0.443526   0.51073418 0.57238189]


In [10]:
interno_std

,Biomass,FossilCoal,FossilGas,FossilOil,Hydro,LoadForecast_nord,WaterReservoirs,WINDforecast,Pvforecast,SECONDARIA,TERZIARIA_TOTALEup,TERZIARIAdw
2018-01-01 00:00:00,-1.986191,-0.142890,-1.618555,-0.772366,-1.419274,-1.493925,0.379475,-0.499797,-0.667226,-1.586280,-0.936565,-0.942864
2018-01-01 01:00:00,-1.069895,0.148684,-1.453557,-0.772366,-1.380383,-1.659326,0.379475,-0.371541,-0.667226,-1.793062,-1.034186,-1.039894
2018-01-01 02:00:00,-2.010304,-0.147709,-1.679179,-0.772366,-1.430635,-1.767168,0.379475,-0.371541,-0.667226,-1.931380,-1.107034,-1.112300
2018-01-01 03:00:00,-1.045782,0.083622,-1.611367,-0.772366,-1.514534,-1.833453,0.379475,-0.243284,-0.667226,-2.017856,-1.160363,-1.165307
2018-01-01 04:00:00,-1.021669,0.037838,-1.590430,-0.772366,-1.704617,-1.836778,0.379475,-0.115028,-0.667226,-2.022224,-1.183993,-1.188794
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,1.148505,0.314953,0.521105,-0.571635,-0.304993,-0.821725,-0.811046,-0.628053,-0.667226,-0.802709,-0.554117,-0.562700
2022-12-31 20:00:00,1.172618,0.314953,0.298608,-0.571635,-0.628353,-1.068995,-0.811046,-0.628053,-0.667226,-1.081344,-0.700112,-0.707807
2022-12-31 21:00:00,1.172618,0.317363,0.003299,-0.571635,-0.887041,-1.308576,-0.811046,-0.628053,-0.667226,-1.361654,-0.827829,-0.834746
2022-12-31 22:00:00,1.003827,0.312543,-0.127949,-0.571635,-1.050032,-1.516989,-0.811046,-0.628053,-0.667226,-1.614744,-0.926174,-0.932492


In [11]:
# (2)
# Dati interni e di perimetro
pca_int = PCA(n_components=5)
prcomp_interno=pca_int.fit(interno_std.values) # componenti principali
scores_interno = pca_int.fit_transform(interno_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_per = PCA(n_components=5)
prcomp_perimetro=pca_per.fit(perimetro_std.values)
scores_perimetro = pca_per.fit_transform(perimetro_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVinterno=prcomp_interno.explained_variance_ratio_
EVinterno_cum=np.cumsum(EVinterno)
EVperimetro=prcomp_perimetro.explained_variance_ratio_
EVperimetro_cum=np.cumsum(EVperimetro)

print("EVinterno_cum: ")
print(EVinterno_cum)
print("EVperimetro_cum: ")
print(EVperimetro_cum)

#fig = go.Figure()
#fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10],y=EVinterno,name="EV interno",yaxis="y1", marker=dict(color='#0a77b4')))
#fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10],y=EVperimetro,name="EV perimetro",yaxis="y1",marker=dict(color='#5f77b8')))
#fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10],y=EVinterno_cum,name="EV interno cumulata",yaxis="y2",marker=dict(color='#f94830')))
#fig.add_trace(go.Scatter(x=[1,2,3,4,5,6,7,8,9,10],y=EVperimetro_cum,name="EV perimetro cumulata",yaxis="y2",marker=dict(color='#d11728')))
## Create axis objects
#fig.update_layout(xaxis=dict(domain=[0,1]),yaxis=dict(title="Explained Variance",titlefont=dict(color="#1f77b4"),tickfont=dict(color="#1f77b4")),
#                  yaxis2=dict(title="Explained Variance cumulated",titlefont=dict(color="#d62728"),tickfont=dict(color="#d62728"),anchor="x",overlaying="y",side="right"))
## Update layout properties
#fig.update_layout(title_text="Explained Variance plot")
#
#fig.write_html('EV_interno_perimetro.html')

EVinterno_cum: 
[0.33666    0.49818896 0.6393506  0.74886415 0.82629165]
EVperimetro_cum: 
[0.22888329 0.38588418 0.48671061 0.57446425 0.63987925]


In [12]:
# (3)
# prod e cons + scambi comm / riserve + transfer capacity
pca_trea = PCA(n_components=5)
prcomp_trea=pca_trea.fit(tre_a_std.values) # componenti principali
scores_trea = pca_trea.fit_transform(tre_a_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_treb = PCA(n_components=5)
prcomp_treb=pca_treb.fit(tre_b_std.values)
scores_treb = pca_treb.fit_transform(tre_b_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVtrea=prcomp_trea.explained_variance_ratio_
EVtrea_cum=np.cumsum(EVtrea)
EVtreb=prcomp_treb.explained_variance_ratio_
EVtreb_cum=np.cumsum(EVtreb)

print("EV3a_cum: ")
print(EVtrea_cum)
print("EV3b_cum: ")
print(EVtreb_cum)

EV3a_cum: 
[0.16407945 0.29558046 0.41250082 0.50449752 0.57239318]
EV3b_cum: 
[0.29694196 0.5537633  0.65906398 0.74664881 0.82368072]


In [13]:
# (4)
# scambi commerciali / interno / transfer capacity
pca_quata = PCA(n_components=5)
prcomp_quata=pca_quata.fit(quat_a_std.values) # componenti principali
scores_quata = pca_quata.fit_transform(quat_a_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_quatb = PCA(n_components=5)
prcomp_quatb=pca_quatb.fit(quat_b_std.values)
scores_quatb = pca_quatb.fit_transform(quat_b_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
pca_quatc = PCA(n_components=5)
prcomp_quatc=pca_quatc.fit(quat_c_std.values)
scores_quatc = pca_quatc.fit_transform(quat_c_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVquata=prcomp_quata.explained_variance_ratio_
EVquata_cum=np.cumsum(EVquata)
EVquatb=prcomp_quatb.explained_variance_ratio_
EVquatb_cum=np.cumsum(EVquatb)
EVquatc=prcomp_quatc.explained_variance_ratio_
EVquatc_cum=np.cumsum(EVquatc)

print("EV4a_cum: ")
print(EVquata_cum)
print("EV4b_cum: ")
print(EVquatb_cum)
print("EV4c_cum: ")
print(EVquatc_cum)

EV4a_cum: 
[0.2914174  0.48960956 0.60118599 0.69240394 0.76962575]
EV4b_cum: 
[0.33666    0.49818896 0.6393506  0.74886415 0.82629165]
EV4c_cum: 
[0.34411205 0.56701517 0.68637308 0.79332707 0.8589457 ]


In [14]:
# (5)
# transfer capacity / interno + scamb comm
pca_cap = PCA(n_components=5)
prcomp_cap=pca_cap.fit(cap_std.values) # componenti principali
scores_cap = pca_cap.fit_transform(cap_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_cinc = PCA(n_components=5)
prcomp_cinc=pca_cinc.fit(cin_c_std.values)
scores_cinc = pca_cinc.fit_transform(cin_c_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVcap=prcomp_cap.explained_variance_ratio_
EVcap_cum=np.cumsum(EVcap)
EVcinc=prcomp_cinc.explained_variance_ratio_
EVcinc_cum=np.cumsum(EVcinc)

print("EV5a_cum: ")
print(EVcap_cum)
print("EV5b_cum: ")
print(EVcinc_cum)

EV5a_cum: 
[0.34411205 0.56701517 0.68637308 0.79332707 0.8589457 ]
EV5b_cum: 
[0.19214941 0.33723904 0.45130962 0.53696364 0.61617117]


In [15]:
# (6)
# scam comm / interno + transfer capacity
pca_sccomm = PCA(n_components=5)
prcomp_sccomm=pca_sccomm.fit(sccomm_std.values) # componenti principali
scores_sccomm = pca_sccomm.fit_transform(sccomm_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_seic = PCA(n_components=5)
prcomp_seic=pca_seic.fit(sei_c_std.values)
scores_seic = pca_seic.fit_transform(sei_c_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVsccomm=prcomp_sccomm.explained_variance_ratio_
EVsccomm_cum=np.cumsum(EVsccomm)
EVseic=prcomp_seic.explained_variance_ratio_
EVseic_cum=np.cumsum(EVseic)

print("EV6a_cum: ")
print(EVsccomm_cum)
print("EV6b_cum: ")
print(EVseic_cum)

EV6a_cum: 
[0.2914174  0.48960956 0.60118599 0.69240394 0.76962575]
EV6b_cum: 
[0.2467061  0.43107181 0.53942011 0.61698074 0.6719796 ]


In [16]:
skip_plots = True

# PCA su dati interni, scambi commerciali e capacità di transito --> scatter plots
pca_int = PCA(n_components=2)
prcomp_interno=pca_int.fit(interno_std.values) # componenti principali
scores_interno = pca_int.fit_transform(interno_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_sccomm = PCA(n_components=2)
prcomp_sccomm=pca_sccomm.fit(sccomm_std.values)
scores_sccomm = pca_sccomm.fit_transform(sccomm_std.values)
pca_cap = PCA(n_components=2)
prcomp_cap=pca_cap.fit(cap_std.values)
scores_cap = pca_cap.fit_transform(cap_std.values) 

sc_int=pd.DataFrame(scores_interno,index=dates,columns=['PC1','PC2'])
load_int=pd.DataFrame(prcomp_interno.components_,index=['PC1','PC2'],columns=(pro_cons_col+res_col))
sc_sccomm=pd.DataFrame(scores_sccomm,index=dates,columns=['PC1','PC2'])
load_sccomm=pd.DataFrame(prcomp_sccomm.components_,index=['PC1','PC2'],columns=scamb_comm_col)
sc_cap=pd.DataFrame(scores_cap,index=dates,columns=['PC1','PC2'])
load_cap=pd.DataFrame(prcomp_cap.components_,index=['PC1','PC2'],columns=cap_col)


if not skip_plots:
    # Scatter plot of principal components (interno)
    fig = go.Figure(data=go.Scattergl(x = sc_int['PC1'],y = sc_int['PC2'],mode='markers',marker=dict(color=dates.hour,colorscale='YlOrRd',line_width=0.5,showscale=True)),
                    layout=dict(title='PCA Analysis - internal data - groupby(hour)',xaxis=dict(title='First Principal Component'),yaxis=dict(title='Second Principal Component')))
    fig.show()
    # fig.write_html('PCA interno per ora.html')

    fig=go.Figure()
    for i in load_int.columns:
        fig.add_trace(go.Scatter(x=[0,load_int[i]['PC1']], y=[0,load_int[i]['PC2']], mode='lines',name=i, text=["",i]))
    fig.show()
    # fig.write_html('PC loading interno.html')

    # Scatter plot of principal components (scambi commerciali)
    fig = go.Figure(data=go.Scattergl(x = sc_sccomm['PC1'],y = sc_sccomm['PC2'],mode='markers',marker=dict(color=dates.hour,colorscale='YlOrRd',line_width=0.5,showscale=True)),
                    layout=dict(title='PCA Analysis - perimeter data - groupby(hour)',xaxis=dict(title='First Princiale Component'),yaxis=dict(title='Second Principal Component')))
    fig.show()
    # fig.write_html('PCA scambi commerciali per ora.html')

    fig=go.Figure()
    for i in load_sccomm.columns:
        fig.add_trace(go.Scatter(x=[0,load_sccomm[i]['PC1']], y=[0,load_sccomm[i]['PC2']], mode='lines',name=i, text=["",i]))
    fig.show()
    # fig.write_html('PC loading scamb comm.html')

    # Scatter plot of principal components (capacità di transito)
    fig = go.Figure(data=go.Scattergl(x = sc_cap['PC1'],y = sc_cap['PC2'],mode='markers',marker=dict(color=dates.hour,colorscale='YlOrRd',line_width=0.5,showscale=True)),
                    layout=dict(title='PCA Analysis - perimeter data - groupby(hour)',xaxis=dict(title='First Principal Component'),yaxis=dict(title='Second Principal Component')))
    fig.show()
    # fig.write_html('PCA capacità di transito per ora.html')

    fig=go.Figure()
    for i in load_cap.columns:
        fig.add_trace(go.Scatter(x=[0,load_cap[i]['PC1']], y=[0,load_cap[i]['PC2']], mode='lines',name=i, text=["",i]))
    fig.show()
    # fig.write_html('PC loading cap trans.html')



In [17]:
skip_plots = True

# PCA su dati interni e di perimetro
pca_int = PCA(n_components=2)
prcomp_interno=pca_int.fit(interno_std.values) # componenti principali
scores_interno = pca_int.fit_transform(interno_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_per = PCA(n_components=2)
prcomp_perimetro=pca_per.fit(perimetro_std.values)
scores_perimetro = pca_per.fit_transform(perimetro_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali

sc_int=pd.DataFrame(scores_interno,index=dates,columns=['PC1','PC2'])
load_int=pd.DataFrame(prcomp_interno.components_,index=['PC1','PC2'],columns=(pro_cons_col+res_col))

sc_per=pd.DataFrame(scores_perimetro,index=dates,columns=['PC1','PC2'])
load_per=pd.DataFrame(prcomp_perimetro.components_,index=['PC1','PC2'],columns=(scamb_comm_col+cap_col))

if not skip_plots:
    # Scatter plot of principal components (interno e perimetro)
    fig = go.Figure(data=go.Scattergl(x = sc_int['PC1'],y = sc_int['PC2'],mode='markers',marker=dict(color=dates.hour,colorscale='YlOrRd',line_width=0.5,showscale=True)),
                    layout=dict(title='PCA Analysis - internal data - groupby(hour)',xaxis=dict(title='First Principle Component'),yaxis=dict(title='Second Principle Component')))
    fig.show()
    #fig.write_html('PCA interno per ora.html')

    fig=go.Figure()
    for i in load_int.columns:
        fig.add_trace(go.Scatter(x=[0,load_int[i]['PC1']], y=[0,load_int[i]['PC2']], mode='lines',name=i, text=["",i]))
    fig.show()
    #fig.write_html('PCA loading interno.html')

    # Scatter plot of principal components (interno e perimetro)
    fig = go.Figure(data=go.Scattergl(x = sc_per['PC1'],y = sc_per['PC2'],mode='markers',marker=dict(color=dates.hour,colorscale='YlOrRd',line_width=0.5,showscale=True)),
                    layout=dict(title='PCA Analysis - perimeter data - groupby(hour)',xaxis=dict(title='First Principle Component'),yaxis=dict(title='Second Principle Component')))
    fig.show()
    #fig.write_html('PCA perimetro per ora.html')

    fig=go.Figure()
    for i in load_per.columns:
        fig.add_trace(go.Scatter(x=[0,load_per[i]['PC1']], y=[0,load_per[i]['PC2']], mode='lines',name=i, text=["",i]))
    fig.show()
    #fig.write_html('PC loading perimetro.html')



In [18]:
skip = True

if not skip:
    scores=sc_cap
    hier_clust = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage='ward').fit(scores.values)
    def plot_dendrogram(model, **kwargs):
        # Create linkage matrix and then plot the dendrogram

        # create the counts of samples under each node
        counts = np.zeros(model.children_.shape[0])
        n_samples = len(model.labels_)
        for i, merge in enumerate(model.children_):
            current_count = 0
            for child_idx in merge:
                if child_idx < n_samples:
                    current_count += 1  # leaf node
                else:
                    current_count += counts[child_idx - n_samples]
            counts[i] = current_count

        linkage_matrix = np.column_stack([model.children_, model.distances_,
                                        counts]).astype(float)

        # Plot the corresponding dendrogram
        dendrogram(linkage_matrix, **kwargs)

    plot_dendrogram(hier_clust, truncate_mode='level', p=6)
else:
    print("Skipped cell")

Skipped cell


In [19]:
skip_plots = True

scores=sc_cap
reduced= cap_std
# K-mean clustering and clusters plot
km_clust=KMeans(n_clusters=2, n_init=10)
km_clust.fit(reduced)
Z = km_clust.predict(reduced)

if not skip_plots:

    # clusters plot
    trace0 = go.Scatter(x=scores['PC1'],y=scores['PC2'],text=scores.index,name='',mode='markers',
                        marker=dict(opacity=0.5,color=Z, line=dict(width=0.2,color='DarkSlateGrey')),
                        showlegend=True)
    # cluster centers
    trace1 = go.Scatter(x=km_clust.cluster_centers_[:, 0],y=km_clust.cluster_centers_[:, 1],name='',mode='markers',
                        marker=go.scatter.Marker(symbol='x',size=12,color='Red'),
                        showlegend=False)
    layout = go.Layout(title='clusters plot',hovermode='closest')
    fig = go.Figure(data=[trace0, trace1], layout=layout)
    fig.show()

In [20]:
cap_std

,CapNord_Cnord,CapCnord_Nord,CapAT_Nord,CapNord_AT,CapCH_Nord,CapNord_CH,CapFR_Nord,CapNord_FR,CapSl_Nord,CapNord_Sl
2018-01-01 00:00:00,0.247352,0.433064,-1.288093,1.380170,-0.956119,1.068744,-1.665647,1.039242,-1.395708,0.460185
2018-01-01 01:00:00,0.247352,0.433064,-1.288093,1.380170,-0.956119,1.068744,-1.665647,1.039242,-1.395708,0.460185
2018-01-01 02:00:00,0.247352,0.433064,-1.288093,1.380170,-0.956119,1.068744,-1.665647,1.039242,-1.395708,0.460185
2018-01-01 03:00:00,0.247352,0.433064,-1.288093,1.380170,-0.956119,1.068744,-1.665647,1.039242,-1.395708,0.460185
2018-01-01 04:00:00,0.247352,0.433064,-1.463572,1.380170,-1.202705,1.068744,-1.952221,1.039242,-1.671216,0.460185
...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,0.247352,1.193685,1.043269,-0.001058,-1.246661,0.501478,1.899392,3.690650,1.162581,0.240215
2022-12-31 20:00:00,0.027283,1.345203,1.043269,-0.001058,-1.246661,0.501478,1.899392,3.690650,1.162581,0.240215
2022-12-31 21:00:00,0.355187,1.523994,1.043269,-0.001058,-1.246661,0.501478,1.899392,3.690650,1.162581,0.240215
2022-12-31 22:00:00,0.355187,1.523994,1.043269,-0.001058,-1.298123,0.501478,1.899392,3.690650,1.162581,0.240215


In [21]:
# (4)
# scambi commerciali / interno / transfer capacity
pca_quata = PCA(n_components=2)
prcomp_quata=pca_quata.fit(quat_a_std.values) # componenti principali
scores_quata = pca_quata.fit_transform(quat_a_std.values) # scores dei campioni (coordinate dei campioni nello spazio delle componenti principali)
pca_quatb = PCA(n_components=2)
prcomp_quatb=pca_quatb.fit(quat_b_std.values)
scores_quatb = pca_quatb.fit_transform(quat_b_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
pca_quatc = PCA(n_components=2)
prcomp_quatc=pca_quatc.fit(quat_c_std.values)
scores_quatc = pca_quatc.fit_transform(quat_c_std.values) # il risultato di .fit_transform sono gli scores, ergo le coordinate dei campioni nello spazio delle componenti principali
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#print(prcomp.explained_variance_ratio_)

## Explained Variance plot
EVquata=prcomp_quata.explained_variance_ratio_
EVquata_cum=np.cumsum(EVquata)
EVquatb=prcomp_quatb.explained_variance_ratio_
EVquatb_cum=np.cumsum(EVquatb)
EVquatc=prcomp_quatc.explained_variance_ratio_
EVquatc_cum=np.cumsum(EVquatc)

print("EV4a: ")
print(EVquata)
print("EV4b: ")
print(EVquatb)
print("EV4c: ")
print(EVquatc)

EV4a: 
[0.2914174  0.19819216]
EV4b: 
[0.33666    0.16152896]
EV4c: 
[0.34411205 0.22290313]


In [22]:
data= np.concatenate((scores_quata,scores_quatb,scores_quatc),axis=1)
columns=['SC_PC1','SC_PC2','IN_PC1','IN_PC2','CT_PC1','CT_PC2']
PC_4=pd.DataFrame(data=data, index=dates, columns=columns)

In [24]:
PC_4.to_pickle('../data/2_processed/PC_strut.pkl')